In [20]:
!pip install dualcodec

Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple


In [21]:
import dualcodec

In [22]:
!huggingface-cli download facebook/w2v-bert-2.0 --local-dir w2v-bert-2.0                        
!huggingface-cli download amphion/dualcodec --local-dir dualcodec_ckpts

Returning existing local_dir `w2v-bert-2.0` as remote repo cannot be accessed in `snapshot_download` ((ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')), '(Request ID: 60160e64-3002-4669-9c90-11c06014059a)')).
/Users/lijiaqi18/github/DualCodec/w2v-bert-2.0
Returning existing local_dir `dualcodec_ckpts` as remote repo cannot be accessed in `snapshot_download` ((ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')), '(Request ID: a704e06e-bdfb-4aa8-ab94-ed89dd1ad721)')).
/Users/lijiaqi18/github/DualCodec/dualcodec_ckpts


In [23]:

w2v_path = "./w2v-bert-2.0"
dualcodec_model_path = "./dualcodec_ckpts"

In [24]:
dualcodec_model = dualcodec.get_model("25hz_v1", dualcodec_model_path) # using the 12hz_v1 model
inference = dualcodec.Inference(dualcodec_model=dualcodec_model, dualcodec_path=dualcodec_model_path, w2v_path=w2v_path, device='cpu')

/Users/lijiaqi18/miniconda3/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loading model from ./dualcodec_ckpts/dualcodec_25hz_16384_1024.safetensors
Model loaded


In [25]:
# download example_wav_en.wav
! curl -o example_wav_en.wav https://fireredteam.github.io/demos/firered_tts/audios/ZeroShotICL_samples/english_to_mix/p276%23p276_354.wav


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  106k  100  106k    0     0   156k      0 --:--:-- --:--:-- --:--:--  156k


In [26]:
! curl -o example_wav_zh.wav https://fireredteam.github.io/demos/firered_tts/audios/SpeakerFinetune_samples/wukong/wukong_000099.wav


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  117k  100  117k    0     0   216k      0 --:--:-- --:--:-- --:--:--  216k


In [8]:
# do inference for English example wav
import torchaudio
audio, sr = torchaudio.load('example_wav_en.wav')
# resample
audio = torchaudio.functional.resample(audio, sr, 24000)
audio = audio.reshape(1,1,-1)
# extract codes
semantic_codes, acoustic_codes = inference.encode(audio, n_quantizers=8)
# produce output audio
out_audio = dualcodec_model.decode_from_codes(semantic_codes, acoustic_codes)

In [9]:
from IPython.display import Audio, display
# Play the input audio
print("Input audio")
display(Audio(audio.cpu().numpy().squeeze(), rate=24000))
# Play the output audio
print("Output audio (8vq)")
Audio(out_audio.cpu().numpy().squeeze(), rate=24000)

Input audio


Output audio (8vq)


In [10]:
# see the shape of codes
print("semantic_codes shape:", semantic_codes.shape)
print("max semantic code:", semantic_codes.max()) # the codebook size for semantic code is 16384
print("acoustic_codes shape:", acoustic_codes.shape)
print("max acoustic code:", acoustic_codes.max()) # the codebook size for acoustic code is 1024(25hz_v1) and 4096(12hz_v1)

semantic_codes shape: torch.Size([1, 1, 28])
max semantic code: tensor(15920)
acoustic_codes shape: torch.Size([1, 7, 28])
max acoustic code: tensor(4058)


In [ ]:
# see the difference with different number of quantizers
for n_q in [1, 2, 4, 8]:
    # decode from codes
    out_audio = dualcodec_model.decode_from_codes(semantic_codes, acoustic_codes[:, :n_q])
    print(f"n_quantizers={n_q}")
    display(Audio(out_audio.cpu().numpy().squeeze(), rate=24000))

n_quantizers=1


n_quantizers=2


n_quantizers=4


n_quantizers=8


In [12]:
# for ZH example
audio, sr = torchaudio.load('example_wav_zh.wav')
# resample
audio = torchaudio.functional.resample(audio, sr, 24000)
audio = audio.reshape(1,1,-1)
# extract codes
semantic_codes, acoustic_codes = inference.encode(audio, n_quantizers=8)
# produce output audio
out_audio = dualcodec_model.decode_from_codes(semantic_codes, acoustic_codes)

In [13]:
print("Input audio")
display(Audio(audio.cpu().numpy().squeeze(), rate=24000))
print("Output audio (8vq)")
display(Audio(out_audio.cpu().numpy().squeeze(), rate=24000))

Input audio


Output audio (8vq)


In [ ]:
# see the difference with different number of quantizers
for n_q in [1, 2, 4, 8]:
    out_audio = dualcodec_model.decode_from_codes(semantic_codes, acoustic_codes[:, :n_q])
    print(f"n_quantizers={n_q}")
    display(Audio(out_audio.cpu().numpy().squeeze(), rate=24000))

n_quantizers=1


n_quantizers=2


n_quantizers=4


n_quantizers=8
